# **1. Data Loading & Preprocessing**

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# 📥 Load Amazon Polarity dataset
dataset = load_dataset("amazon_polarity")

# 🔤 Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# ✂️ Tokenization function with truncation and padding
def tokenize_fn(example):
    return tokenizer(
        example["content"],
        truncation=True,
        padding="max_length",  # ensures fixed length
        max_length=128         # can change to 256 if needed
    )

# 🧼 Apply tokenization
# Removed fn_kwargs=dict(tokenizer=tokenizer) as tokenize_fn doesn't accept it
tokenized_dataset = dataset.map(tokenize_fn, batched=True, num_proc=96)

# 🏷️ Rename label column for Trainer compatibility
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# 🔁 Format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 🔀 Split train into train + validation (90/10)
train_val_split = tokenized_dataset["train"].train_test_split(test_size=0.1)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = tokenized_dataset["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

amazon_polarity/train-00000-of-00004.par(…):   0%|          | 0.00/260M [00:00<?, ?B/s]

amazon_polarity/train-00001-of-00004.par(…):   0%|          | 0.00/258M [00:00<?, ?B/s]

amazon_polarity/train-00002-of-00004.par(…):   0%|          | 0.00/255M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map (num_proc=96):   0%|          | 0/3600000 [00:00<?, ? examples/s]

# **2. Model Setup**

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


# **3. Training Setup**

In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 🧮 Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ⚙️ Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
)

# 🏋️ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# **4. Fine-Tuning**

In [ ]:
# Use a smaller subset for quick training
small_train = train_dataset.select(range(10000))
small_val = val_dataset.select(range(5000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val,
    compute_metrics=compute_metrics
)

trainer.train()


# **5. Plotting Loss**

In [ ]:
import matplotlib.pyplot as plt

# Extract logs
logs = trainer.state.log_history
train_loss = [log["loss"] for log in logs if "loss" in log]
eval_loss = [log["eval_loss"] for log in logs if "eval_loss" in log]

# Plot
plt.plot(train_loss, label="Train Loss")
plt.plot(eval_loss, label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss vs Epochs")
plt.show()
